<a href="https://colab.research.google.com/github/Mestevam1976/Imersao_IA_Alura/blob/main/%5BImersao_IA_2%C2%AA_ed_%5D_Criando_um_sistema_para_busca_em_documentos_usando_embeddings_e_a_Gemini_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><font color='red'><b>[Imersão IA - 2ª ed.] Criando um sistema para busca em documentos usando embeddings e a Gemini API - Aula 5</b></font></h1>

<h2><font color='blue'><b>Márcio Estevam da Silva - Analista de Business Intelligence</b></font></h2>

<hr>

<h3><b>Instalação do SDK da Google</b> <img src='https://cdn.iconscout.com/icon/premium/png-512-thumb/sdk-7992814-6673727.png?f=webp&w=256' width=50 height=50></p>

In [1]:
!pip install -U -q google-generativeai

<h3><b>Importações de bibliotecas e configurações iniciais:</b> <img src='https://cdn.iconscout.com/icon/premium/png-512-thumb/lib-8813868-7123596.png?f=webp&w=256' width=50 height=50</h3>

In [2]:
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata


GOOGLE_API_KEY = userdata.get('SECRET_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

***Exibindo os modelos disponíveis da IA:***

In [3]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


<hr>

<h3><b>O que são Embeddings?</b> <img src='https://cdn.iconscout.com/icon/premium/png-512-thumb/embedding-1573539-1333034.png?f=webp&w=256' width=50 height=50</h3>

<p>Embeddings de texto são uma técnica de processamento de linguagem natural (PLN) que converte texto em vetores numéricos. Os embeddings capturam significado semântico e contexto, resultando em um texto com significados semelhantes com embeddings mais próximos. Por exemplo, as frases "Levei meu cachorro ao veterinário" e "Levei meu gato ao veterinário" teriam embeddings próximos uns dos outros no espaço vetorial, já que ambas descrevem um contexto semelhante.</p>

<p>Isso é importante porque desbloqueia muitos algoritmos que podem operar em vetores, mas não diretamente no texto.</p>

<p>É possível usar esses embeddings ou vetores para comparar diferentes textos e entender como eles se relacionam. Por exemplo, se os embeddings do texto "gato" e "cão" estiverem próximos, você poderá inferir que essas palavras são semelhantes em significado, contexto ou ambos.</p>
<br>
<p>Fonte: <font color='green'>https://ai.google.dev/gemini-api/docs/embeddings?hl=pt-br</font></p>

In [4]:
# Exemplo de embedding
title = "A próxima geração de IA para desenvolvedores e Google Workspace"
sample_text = ("Título: A próxima geração de IA para desenvolvedores e Google Workspace"
    "\n"
    "Artigo completo:\n"
    "\n"
    "Gemini API & Google AI Studio: Uma maneira acessível de explorar e criar protótipos com aplicações de IA generativa")

embedding = genai.embed_content(model="models/embedding-001",
                                content=sample_text,
                                title=title,
                                task_type="RETRIEVAL_DOCUMENT")

print(embedding)

{'embedding': [0.051496185, -0.0365346, -0.0186796, 0.019455563, 0.06721682, -0.003868702, -0.027666211, -0.01814636, 0.062080767, 0.062902406, 0.012042811, 0.01314806, -0.045869622, -0.022034725, 0.009636108, -0.024947647, 0.021764254, -0.011328052, -0.03484635, -0.00094051583, 0.009192941, 0.0068539544, -0.029636258, -0.06623153, -0.019914951, 0.020645097, 0.015273816, -0.038156737, -0.032827046, 0.02755795, -0.049995217, 0.05074531, -0.034130614, 0.010284468, -0.036612358, -0.040648427, -0.026028484, -0.044465132, -0.002034308, -0.0036913794, 0.0061324253, -0.084707916, -0.009676676, 0.02611583, -0.0035269416, -0.021471972, 0.04472568, 0.04406418, 0.01429429, -0.058487777, 0.027538216, 0.023262272, 0.066298164, -0.041965708, -0.0041519217, -0.009309915, 0.022716599, -0.032830838, 0.02376307, 0.0006589484, -0.0012957085, 0.023344554, -0.016953444, 0.052708756, 0.01914495, -0.05771819, -0.04336735, -9.953024e-05, 0.011018355, 0.048855364, 0.009574091, 0.012556762, 0.060748216, -0.0642

<hr>

<h3><b>Aplicando o Embedding</b> <img src='https://cdn.iconscout.com/icon/premium/png-512-thumb/embedding-1886484-1598680.png?f=webp&w=256' width=50 heigth=50</h3>

In [5]:
DOCUMENT1 = {
    "Título":"Operação do sistema de controle climático",
    "Conteúdo": "O Googlecar tem um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema."}
DOCUMENT2 = {
    "Título": "Touchscreen",
    "Conteúdo":"O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos."}
DOCUMENT3 = {
    "Título": "Mudança de marchas",
    "Conteúdo":"Seu Googlecar tem uma transmissão automática.Para trocar de marcha basta apertar o botão correspondente."}

documents =  [DOCUMENT1, DOCUMENT2, DOCUMENT3]

<p><b>Gerando um DataFrame com os documentos</b></p>

In [6]:
df = pd.DataFrame(documents)
df

,Título,Conteúdo
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática.P...


<p><b>Seleção do Modelo</b> <img src='https://cdn.iconscout.com/icon/premium/png-512-thumb/selection-2348580-1954463.png?f=webp&w=256' width=50 heigth=50</p>

In [7]:
model = "models/embedding-001"

<p><b>Criando uma função para executar o embedding em um DataFrame</b></p>

In [8]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                content=text,
                                title=title,
                                task_type="RETRIEVAL_DOCUMENT")["embedding"]

<p><b>Criando uma nova coluna no DataFrame e aplicando a função "embed_fn"</b></p>

In [9]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Título"],row["Conteúdo"]),axis=1)

df

,Título,Conteúdo,Embeddings
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...,"[-0.010693488, -0.012540616, -0.05604656, 0.04..."
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...,"[0.015988398, -0.043456998, -0.022340337, -0.0..."
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática.P...,"[-0.0011369474, -0.04646206, -0.013126202, 0.0..."


<p><b>Criando uma função melhorada, que aplica uma busca utilizando o prompt do Google AI Studio</b></p>

In [10]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_consulta = genai.embed_content(model=model,
                                content=consulta,
                                title=title,
                                task_type="RETRIEVAL_DOCUMENT")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteúdo"]

<p><b>Executando uma consulta:</b> <img src='https://cdn.iconscout.com/icon/premium/png-512-thumb/query-5469393-4552478.png?f=webp&w=256' width=50 height=50</p>

In [11]:
consulta = "Como fazer para trocar de marcha?"

trecho = gerar_e_buscar_consulta(consulta, df, model)

print(trecho)

Seu Googlecar tem uma transmissão automática.Para trocar de marcha basta apertar o botão correspondente.


<p><b>Ajustando as configurações para obter respostas mais personalizadas</b></p>

In [12]:
generation_config = {
    "temperature":1,
    "candidate_count": 1
}

In [13]:
prompt = f'Reescreva esse texto de uma forma mais descontraída, sem adicionar informações que não façam parte do texot: {trecho}'

model_2 = genai.GenerativeModel("gemini-1.0-pro", generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

Ei, seu carrinho doidão do Google tem uma transmissão de boa vida! Nada de ficá suando trocando marchas, é só tacar o dedo no botaõo!


In [14]:
pwd

'/content'